In [7]:
import requests
import json
import pandas as pd
import time
from config import API_key
import logging

In [8]:
logging.basicConfig(level=logging.INFO, filename="API_google_log.log",filemode="w",
                    format="%(asctime)s %(levelname)s %(message)s")

In [9]:
'''
with open('scraping_log.log', 'w'):
    pass # Clearing logs
'''

"\nwith open('scraping_log.log', 'w'):\n    pass # Clearing logs\n"

In [10]:
all_books = pd.read_csv("All_added_books_data.csv")

In [ ]:
titles = all_books['Title'].dropna().tolist()

base_url = "https://www.googleapis.com/books/v1/volumes"
results = []

for title in titles:
    logging.info(f"Started getting info about '{title}' title")
    search_query = title.replace(' ', '+')
    url = f"{base_url}?q=intitle:{search_query}"
    flag = True
    while flag:
      try:
          response = requests.get(url, headers={'accept': 'application/json', 'X-API-KEY': API_key})
          if response.status_code == 200:
              logging.info(f"Response 200 for '{title}' title")
              flag = False
              response_data = response.json()

              if 'items' in response_data and len(response_data['items']) > 0:
                  logging.info(f"Got data for '{title}' title")
                  first_result = response_data['items'][0]['volumeInfo']
                  results.append({
                      "Title": title,
                      "API Title": first_result.get('title'),
                      "Authors": first_result.get('authors', []),
                      "Publisher": first_result.get('publisher'),
                      "Published Date": first_result.get('publishedDate'),
                      "Description": first_result.get('description'),
                      "Page Count": first_result.get('pageCount'),
                      "Print Type": first_result.get('printType'),
                      "Categories": first_result.get('categories', []),
                      "Average Rating": first_result.get('averageRating'),
                      "Ratings Count": first_result.get('ratingsCount'),
                      "Language": first_result.get('language'),
                  })
              else:
                  logging.info(f"No data for '{title}' title")
                  results.append({
                      "Title": title,
                      "API Title": None,
                      "Authors": None,
                      "Publisher": None,
                      "Published Date": None,
                      "Description": None,
                      "Page Count": None,
                      "Print Type": None,
                      "Categories": None,
                      "Average Rating": None,
                      "Ratings Count": None,
                      "Language": None,
                  })
          else:
              logging.warning(f"Error appeared while requesting data for '{title}' title: {response.status_code}")
              time.sleep(1)
      except Exception as e:
          logging.error(f"Error appeared while requesting data for '{title}'", exc_info=e)


    time.sleep(0.2)

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("all_api_books.csv", index=False)